Para este endpoint se nos pide devolver el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos). Para esto voy a usar la columna sentiment_analysis creada a partir del modelo de machine learning aplicada a la columna clean_reviews. Sentiment analysis tiene valores 0 para reviews malas, 1 para reviews neutrales o nulas y 2 para reviews buenas.

In [1]:
import pandas as pd
import pyarrow.parquet as pq

In [2]:
df_reviews = pq.read_table('Datos/df_reviews_sentimiento.parquet').to_pandas()
df_reviews.head()

,user_id,posted,item_id,recommend,review,clean_reviews,sentiment_analysis
0,76561197970982479,2011,1250,true,simple yet with great replayability. in my opi...,simple yet great replayability opinion zombie ...,2
1,76561197970982479,2011,22200,true,its unique and worth a playthrough.,unique worth playthrough,2
2,76561197970982479,2011,43110,true,great atmosphere. the gunplay can be a bit chu...,great atmosphere gunplay bit chunky time end d...,2
3,js41637,2014,251610,true,i know what you think when you see this title ...,know think see title barbie dreamhouse party i...,2
4,js41637,2013,227300,true,for a simple (its actually not all that simple...,simple actually simple truck driving simulator...,0


In [4]:
df_reviews.shape

(59305, 7)

Selecciono las columnas que voy a usar de esta tabla:

In [6]:
df_endpoint2 = df_reviews[['item_id','sentiment_analysis']]

Sólo voy a usar los valores de sentiment analysis 0 o 2, ya que los neutrales ni los nulos me importan para el análisis.

In [7]:
df_endpoint2 = df_endpoint2[(df_endpoint2['sentiment_analysis'] == 0) | (df_endpoint2['sentiment_analysis'] == 2)]

In [17]:
df_endpoint2['sentiment_analysis'].value_counts()

sentiment_analysis
2    31133
0    14232
Name: count, dtype: int64

Como tengo que usar también la tabla steam_games, la voy a traer para ver qué datos necesito y poder seguir con la actividad

In [18]:
df_steam_games = pq.read_table('df_endpoint1.parquet').to_pandas()

Me quedo con las columnas que voy a usar

In [19]:
df_steam_games.drop(columns=['price','anio'], axis=1, inplace=True)

In [20]:
df_steam_games.head()

,id,developer
0,0,none
1,10,valve
2,20,valve
3,30,valve
4,40,valve


In [21]:
df_endpoint2.head()

,item_id,sentiment_analysis
0,1250,2
1,22200,2
2,43110,2
3,251610,2
4,227300,0


Hago el marge (innter join, ya que quiero solo los resultados que coincidan en ambas tablas) entre las dos tablas con el nombre de variable "resultado"

In [22]:
resultado = df_steam_games.merge(df_endpoint2, left_on='id', right_on='item_id', how='inner')

In [23]:
resultado.head()

,id,developer,item_id,sentiment_analysis
0,10,valve,10,0
1,10,valve,10,0
2,10,valve,10,2
3,10,valve,10,2
4,10,valve,10,0


In [15]:
resultado.shape

(40781, 4)

La columna 'developer' tiene algunos caracteres que quiero quitar:

In [25]:
resultado['developer'] = resultado['developer'].str.replace('   ',' ').str.replace('.','')

In [26]:
!pip install unidecode
from unidecode import unidecode
resultado['developer'] = resultado['developer'].apply(unidecode)

     -------------------------------------- 235.5/235.5 kB 1.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Limpieza general de resultado

In [27]:
# Reemplaza '-' por ' ' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('-', ' ') if x.dtype == "object" else x)

# Reemplaza '!' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('!', '') if x.dtype == "object" else x)

# Reemplaza '¡' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('¡', '') if x.dtype == "object" else x)

# Reemplaza '?' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('?', '') if x.dtype == "object" else x)

# Reemplaza '¿' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('¿', '') if x.dtype == "object" else x)

# Reemplaza '"' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('"', '') if x.dtype == "object" else x)

# Reemplaza ''' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("'", '') if x.dtype == "object" else x)

# Reemplaza ',' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace(",", '') if x.dtype == "object" else x)

# Reemplaza '&' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("&", '') if x.dtype == "object" else x)

# Reemplaza "''" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("''", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("_", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("[", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("]", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace(";", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("(", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace(")", '') if x.dtype == "object" else x)

Siguiendo con la limpieza, hay developers que quiero corregir. Para esto hago una funcion que retire las palabras seleccionadas. Lo hice de esta manera porque no hay tantas filas que quiera cambiar:

In [28]:
words_to_remove = [
    ' incferal interactive macferal interactive linux',
    ' mac amp linux',
    ' interactive mac',
    ' interactive macferal interactive linux',
    ' gamesaspyr macvirtual programming linux',
    ' softwareaspyr mac amp linux',
    ' interactive mac/linux',
    'incferal interactive macferal interactive linux '
]

def clean_developer_name(name):
    for word in words_to_remove:
        name = name.replace(word, '').strip()
    return name

Creo una nueva columna aplicando la función a la columna 'developer'

In [32]:
resultado['cleaned_developer_column'] = resultado['developer'].apply(clean_developer_name)

In [33]:
resultado['developer'] = resultado['cleaned_developer_column']

Reemplazo una por la otra y dropeo la columna nueva. Ahora 'developer' debería estar limpia

In [34]:
resultado.drop('cleaned_developer_column',axis=1, inplace=True)

Dropeo item_id tambien porque ya hay una columna con id

In [35]:
resultado.drop('item_id',axis=1, inplace=True)

In [36]:
resultado.head()

,id,developer,sentiment_analysis
0,10,valve,0
1,10,valve,0
2,10,valve,2
3,10,valve,2
4,10,valve,0


In [44]:
import pyarrow.parquet as pq
resultado.to_parquet('df_endpoint2.parquet')

La variable "cantidad" almacena la cantidad de reviews positivas y negativas por developer. Voy a usar valve nuevamente porque es el primero que veo en el head()

In [47]:
cantidad = resultado[(resultado['developer'] == 'valve')].groupby('sentiment_analysis').count()

In [48]:
cantidad

,id,developer
sentiment_analysis,,
0,1807,1807
2,4982,4982


Nuevamente creo dos listas para devolver el resultado en formato diccionario

In [49]:
cantidad_reviews = cantidad['developer'].tolist()
sentimiento = cantidad.index.tolist()

In [51]:
cantidad_reviews,sentimiento

([1807, 4982], [0, 2])

In [52]:
negativos = str(cantidad_reviews[0])
negativos = 'Negativos: ' + negativos

positivos = str(cantidad_reviews[1])
positivos = 'Positivos: ' + positivos

In [53]:
negativos, positivos

('Negativos: 1807', 'Positivos: 4982')

Ya tengo los valores que quiero, ahora sólo tengo que hacer el diccionario que sea retornado como resultado

In [56]:
dicc = {}
dicc['valve'] = [negativos,positivos]

In [57]:
dicc

{'valve': ['Negativos: 1807', 'Positivos: 4982']}